<a href="https://colab.research.google.com/github/beltima/extract-wikidata-names/blob/get-names-dictionary/Extract_Wiki_Data_names_try_01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [96]:
!pip install qwikidata
import qwikidata

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [97]:
from qwikidata.sparql import get_subclasses_of_item
from qwikidata.sparql import return_sparql_query_results

#look for ID of a name, first get the ID of name = Gregory
name = 'Gregory'
query = f'''
SELECT DISTINCT ?item ?itemLabel WHERE {{
  ?item rdfs:label "{name}"@en;
        wdt:P31 wd:Q12308941.
  SERVICE wikibase:label {{ bd:serviceParam wikibase:language "en". }}
}}

'''
query_result = return_sparql_query_results(query)

In [98]:
query_result

{'head': {'vars': ['item', 'itemLabel']},
 'results': {'bindings': [{'item': {'type': 'uri',
     'value': 'http://www.wikidata.org/entity/Q922983'},
    'itemLabel': {'xml:lang': 'en', 'type': 'literal', 'value': 'Gregory'}}]}}

In [99]:
#extract the ID for a name
item_id = query_result['results']['bindings'][0]['item']['value'].split('/')[-1]
item_id

'Q922983'

In [100]:
#find similar names from the ID just found for the target name
query1 = f"""
SELECT DISTINCT ?item ?itemLabel WHERE {{
  SERVICE wikibase:label {{ bd:serviceParam wikibase:language "en". }}
  {{
    SELECT DISTINCT ?item WHERE {{
      ?item p:P460 ?statement0.
      ?statement0 (ps:P460/(wdt:P279*)) wd:{item_id}.
    }}

  }}
}}
"""
query_result1 = return_sparql_query_results(query1)

In [101]:
similar_names = [x["itemLabel"]["value"] for x in query_result1["results"]["bindings"]]

In [103]:
male_name_list = {name: similar_names}
print(male_name_list)

{'Gregory': ['Grzegorz', 'Gergely', 'Greggory', 'Grigori', 'Gregorius', 'Greger', 'Gregor', 'Grégory', 'Grégoire', 'Greig', 'Gregorio', 'Krikor', 'Gregers', 'Ryhor', 'Řehoř', 'Grigoris', 'Grigalius', 'Reijo', 'Grigorios', 'Hryhorii']}


In [104]:
##UPDATE it worked!!! 
###CODE WORKED PERFECTLY ON QUERY SERVICE BUT NOT ON PYTHON CODE
#find (first name + last name) where first name = gregory
query2 ="""
SELECT DISTINCT ?item ?itemLabel WHERE {
  SERVICE wikibase:label { bd:serviceParam wikibase:language "en". }
  {
    SELECT DISTINCT ?item ?itemLabel WHERE {
      ?item p:P735 ?statement0.
      ?statement0 (ps:P735/(wdt:P279*)) wd:Q922983.
    }
    LIMIT 100
  }
  FILTER REGEX(?itemLabel, "^Gregory [A-Za-z]+$")
}
"""

query_result2 = return_sparql_query_results(query2)

In [108]:
same_first_name_pair = [ x["itemLabel"]["value"] for x in query_result2['results']['bindings']]

In [110]:
name_pairs = {name: same_first_name_pair}
print(name_pairs)

{'Gregory': ['Gregory Peck', 'Gregory Baum', 'Gregory Doran', 'Gregory Ulmer', 'Gregory Maguire', 'Gregory Bicskei', 'Gregory Mathews', 'Gregory Dark', 'Gregory Kaidanov', 'Gregory Campbell', 'Gregory Nelson', 'Gregory Bittman', 'Gregory Pitl', 'Gregory Sierra', 'Gregory Springer', 'Gregory Atta', 'Gregory Bouchelaghem', 'Gregory Cohen', 'Gregory Franchi', 'Gregory Choplin', 'Gregory Dix', 'Gregory Despres', 'Gregory Dexter', 'Gregory Douglass', 'Gregory Floyd', 'Gregory Fulton', 'Gregory Haerr', 'Gregory James', 'Gregory Kealey', 'Gregory Lekhtman', 'Gregory Luna', 'Gregory Mosher', 'Gregory Orfalea', 'Gregory Penske', 'Gregory Pence', 'Gregory Petsko', 'Gregory Perino', 'Gregory Reyes', 'Gregory Rogers', 'Gregory Sica', 'Gregory Sumner', 'Gregory Wright', 'Gregory Leach', 'Gregory Smits', 'Gregory James', 'Gregory Wong', 'Gregory Popdimitrov', 'Gregory Alcidas', 'Gregory Tanyi', 'Gregory Nouhaud', 'Gregory Wendt', 'Gregory Kosteck', 'Gregory Sharbabchyan', 'Gregory DePalma']}


In [89]:
#find female politicians
query3 = '''
SELECT DISTINCT ?item ?itemLabel WHERE {
  SERVICE wikibase:label { bd:serviceParam wikibase:language "en". }
  {
    SELECT DISTINCT ?item ?itemLabel WHERE {
      ?item wdt:P106 wd:Q82955; 
            wdt:P21 wd:Q6581072.
    }
    ORDER BY RAND()
  }
  FILTER REGEX(?itemLabel, "^[A-Za-z]+ [A-Za-z]+$")
}
LIMIT 10
'''
query_result3 = return_sparql_query_results(query3)

In [94]:
female_politicians = [ x["itemLabel"]["value"] for x in query_result3['results']['bindings']]

In [95]:
female_politicians

['Barbara Gysi',
 'Hedwig Dransfeld',
 'Shirley Temple',
 'Gabrielle Giffords',
 'Mary Robinson',
 'Chandrika Kumaratunga',
 'Jehan Sadat',
 'Martha Washington',
 'Barbara Gordon',
 'Constance Markievicz']